### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 08 декабря 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 08 декабря, -4 балла после 08:30 15 декабря, -6 баллов после 08:30 22 декабря, -8 баллов после 08:30 29 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [3]:

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):

        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None

        if criterion == 'entropy':
            self.impurity = self.__entropy
            self.feature = lambda y: - \
                ((np.bincount(y) / y.size) * np.log2(np.bincount(y) / y.size)).sum()
        elif criterion == 'gini':
            self.impurity = self.__gini
            self.feature = lambda y: 1 - ((np.bincount(y) / y.size) ** 2).sum()
        elif criterion == 'misclass':
            self.impurity = self.__misclass
            self.feature = lambda y: 1 - (np.bincount(y) / y.size).max()

        if max_features == 'sqrt':
            self.get_feature = self.get_feature_sqrt
        elif max_features == 'log2':
            self.get_feature = self.get_feature_log2
        elif max_features is None:
            self.get_feature = self.get_feature_n

    def __gini(self, lc, ls, rc, rs):
        ls = ls.astype('float')
        rs = rs.astype('float')
        return 1 - ((lc ** 2 / ls).sum(axis=1) + (rc ** 2 / rs).sum(axis=1)) / (ls[0] + rs[0])

    def __entropy(self, lc, ls, rc, rs):
        return -((lc * np.log2(lc / ls)).sum(axis=1) + (rc * np.log2(rc / rs)).sum(axis=1)) / (ls[0] + rs[0])

    def __misclass(self, lc, ls, rc, rs):
        return 1 - (lc.max(axis=1) + rc.max(axis=1)) / (ls[0] + rs[0])

    def get_feature_sqrt(self, n):
        feature = np.arange(n)
        np.random.shuffle(feature)
        return np.array(feature[: max(1, np.int(np.log2(n)))])

    def get_feature_log2(self, n):
        feature = np.arange(n)
        np.random.shuffle(feature)
        return np.array(feature[: max(1, np.int(np.sqrt(n)))])

    def get_feature_n(self, n):
        return np.arange(n)

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __sort_samples(self, x, y):
        id_x = x.argsort()
        return x[id_x], y[id_x]

    def __find_threshold(self, x, y):

        sorted_x, sorted_y = self.__sort_samples(x, y)
        number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        if cut_size == 0:
            sp_y = sorted_y
        else:
            sp_y = sorted_y[cut_size:-cut_size]
        bord = np.where(sp_y[:-1] != sp_y[1:])[0] + (cut_size + 1)

        if len(bord) == 0:
            return np.inf, None

        eq_el_count = bord - np.append(np.array([cut_size]), bord[:-1])
        one_hot_code = np.zeros((bord.shape[0], number))
        one_hot_code[np.arange(bord.shape[0]), sorted_y[bord - 1]] = 1

        increments = one_hot_code * eq_el_count.reshape(-1, 1)
        increments[0] += np.bincount(sorted_y[:cut_size], minlength=number)

        l_count = np.cumsum(increments, axis=0)
        r_count = np.bincount(sorted_y, minlength=number) - l_count
        l_sizes = bord.reshape(l_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        G = self.impurity(l_count, l_sizes, r_count, r_sizes)
        ind = np.argmin(G)
        left = l_sizes[ind][0]

        return G[ind], (sorted_x[left-1] + sorted_x[left]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        y_class_count = np.bincount(y)
        y_class_max = y_class_count.argmax()
        node_leaf = self.LEAF_TYPE, y_class_max, y_class_count / y.size

        if (y.size < self.min_samples_split) or (depth == self.max_depth):
            self.tree[node_id] = node_leaf
            return
        if y_class_count[y_class_max] >= y.size * self.sufficient_share:
            self.tree[node_id] = node_leaf
            return

        features = self.get_feature(x.shape[1])
        buf = np.array([self.__find_threshold(x[:, i], y) for i in features])
        best_imp = buf[:, :1].argmin()
        threshold = list(buf[best_imp]) + [best_imp]

        Xleft, Xright, Yleft, Yright = self.__div_samples(
            x, y, threshold[2], threshold[1])
        if not Xleft.size or not Xright.size:
            self.tree[node_id] = node_leaf
        else:
            self.tree[node_id] = self.NON_LEAF_TYPE, threshold[2], threshold[1]
            self.feature_importances_[threshold[2]] += self.feature(y) * y.size
            self.feature_importances_[
                threshold[2]] -= (self.feature(Yleft) * Yleft.size + self.feature(Yright) * Yright.size)
            self.__fit_node(Xleft, Yleft, 2 * node_id + 1, depth + 1, pred_f)
            self.__fit_node(Xright, Yright, 2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [4]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [5]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(
    wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [6]:
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [7]:
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)

0.8888888888888888

In [8]:
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

0.8888888888888888

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [9]:
%time clf.fit(X_train, y_train)

Wall time: 0 ns


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [10]:
%time my_clf.fit(X_train, y_train)

Wall time: 8.01 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [11]:
df = pd.read_csv('Speed Dating Data.csv', sep=',', encoding="ISO-8859-1")
df.shape

(8378, 195)

**Посмотрим сколько NAN содержится в каждой из колонок, возьмем те, в которых NAN < 25**

In [12]:
s = np.array([df.loc[(pd.isna(df[a])), a ].shape[0] for a in df.columns])

In [13]:
S = s.argsort()
S[:s[s <= 25].shape[0]]

array([ 0, 23, 14, 12, 10,  9,  7, 97,  5,  4,  3,  2,  6,  1, 11],
      dtype=int64)

In [14]:
cols = df.columns[S[:s[s <= 25].shape[0]]]
cols

Index(['iid', 'dec_o', 'samerace', 'match', 'partner', 'order', 'position',
       'dec', 'wave', 'condtn', 'idg', 'gender', 'round', 'id', 'pid'],
      dtype='object')

In [15]:
df2 = df[['match', 'iid', 'dec_o', 'samerace', 'partner', 'order',
          'position', 'dec', 'wave', 'condtn', 'idg', 'gender', 'round', 'id', 'pid']]
df2 = df2.dropna()

In [16]:
men = df2[df2.gender == 1].drop_duplicates(subset=['iid', 'pid'])
men = men.drop(['gender'], axis=1).dropna()

women = df2[df2.gender == 0].drop_duplicates(subset=['iid'])
women = women.drop(['gender', 'match', 'pid'], axis=1).dropna()
women.columns = women.columns + '_woman'

In [17]:
dfs = men.join(women.set_index('iid_woman'), on='pid', how='inner')
dfs = dfs.drop(['iid', 'pid'], axis=1)
dfs

,match,dec_o,samerace,partner,order,position,dec,wave,condtn,idg,...,samerace_woman,partner_woman,order_woman,position_woman,dec_woman,wave_woman,condtn_woman,idg_woman,round_woman,id_woman
100,0,1,0,1,4,7,0,1,1,2,...,0,1,4,7,1,1,1,1,10,1.0
110,0,1,0,1,3,7,0,1,1,4,...,0,1,4,7,1,1,1,1,10,1.0
120,1,1,1,1,10,7,1,1,1,6,...,0,1,4,7,1,1,1,1,10,1.0
130,1,1,0,1,5,7,1,1,1,8,...,0,1,4,7,1,1,1,1,10,1.0
140,1,1,0,1,7,7,1,1,1,10,...,0,1,4,7,1,1,1,1,10,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8267,0,0,0,22,9,2,0,21,2,34,...,1,1,11,2,1,21,2,43,22,22.0
8289,0,0,0,22,16,2,0,21,2,36,...,1,1,11,2,1,21,2,43,22,22.0
8311,0,1,0,22,18,2,0,21,2,38,...,1,1,11,2,1,21,2,43,22,22.0
8333,0,0,0,22,5,2,0,21,2,40,...,1,1,11,2,1,21,2,43,22,22.0


In [18]:
X = dfs.iloc[:, 1:].values
y = dfs.iloc[:, 0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf = DecisionTreeClassifier(min_samples_split=2)
myclf = MyDecisionTreeClassifier(min_samples_split=2)

In [19]:
%time clf.fit(X_train, y_train)

Wall time: 2.99 ms


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [20]:
%time myclf.fit(X_train, y_train)

Wall time: 12 ms


In [21]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

1.0

In [22]:
f1_score(y_pred=myclf.predict(X_test), y_true=y_test, average='macro')

1.0

## Находим самые важные признаки (2 балла)

По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [23]:
pd.Series(data=myclf.feature_importances_,
          index=dfs.columns[1:]).sort_values(ascending=False).head(10)

dec            0.659288
dec_o          0.340712
round_woman    0.000000
samerace       0.000000
partner        0.000000
order          0.000000
position       0.000000
wave           0.000000
condtn         0.000000
idg            0.000000
dtype: float64

In [24]:
pd.Series(data=clf.feature_importances_,
          index=dfs.columns[1:]).sort_values(ascending=False).head(10)

dec            0.659288
dec_o          0.340712
round_woman    0.000000
samerace       0.000000
partner        0.000000
order          0.000000
position       0.000000
wave           0.000000
condtn         0.000000
idg            0.000000
dtype: float64

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ



Материал доходчиво объяснен на целых трех семинарах это приятно, но пар стало очень много в последнее время))